# Creating Carto Maps Programmatically

In [139]:
import json
import os
import requests

cred = json.load(open('credentials.json')) # modify credentials.json.sample

username = cred['username']
api_key  = cred['api_key']

In [140]:
mapconfig_text = '''{
  "version": "0.0.1",
  "name": "%s",
  "auth": {
    "method": "open"
  },
  "layergroup": {
    "layers": [{
      "type": "mapnik",
      "options": {
        "cartocss_version": "2.1.1",
        "cartocss": "%s",
        "sql": "%s"
      }
    }]
  }
}'''

map_table = 'yellow_pickups_by_tract_2015_04_15_n_3000'
map_name  = 'taxi_test9'
map_css   = '#layer {polygon-fill: ramp([count_pickup], (#d2fbd4, #a5dbc2, #7bbcb0, #559c9e, #3a7c89, #235d72, #123f5a), quantiles); line-width: 0.5; line-color: #FFF; line-opacity: 0.5; polygon-opacity: 0.5;}'
map_sql   = 'select * from '+map_table



In [141]:
text_file = open("mapconfig.json", "w")
text_file.write(mapconfig_text % (map_name, map_css, map_sql))
text_file.close()

In [142]:
sendmapconfig = "curl -X POST -H 'Content-Type: application/json' -d @mapconfig.json 'https://%s.carto.com/api/v1/map/named?api_key=%s'" % (username, api_key)

#print sendmapconfig
os.system(sendmapconfig)


0

In [143]:
html_text = '''<!DOCTYPE html>
    <html>
      <head>
        <title>Named Map with Interactivity | CartoDB.js</title>
        <meta name="viewport" content="initial-scale=1.0, user-scalable=no" />
        <meta http-equiv="content-type" content="text/html; charset=UTF-8"/>
        <link rel="shortcut icon" href="http://cartodb.com/assets/favicon.ico" />
        <style>
          html, body, #map {height: 100%;
            padding: 0;
            margin: 0;
          }
        </style>

        <link rel="stylesheet" href="http://libs.cartocdn.com/cartodb.js/v3/themes/css/cartodb.css" />
        <!--[if lte IE 8]>
            <link rel="stylesheet" href="http://libs.cartocdn.com/cartodb.js/v3/themes/css/cartodb.ie.css" />
        <![endif]-->

      </head>
      <body>
        <div id="map"></div>

        <!-- include cartodb.js library -->
        <script src="http://libs.cartocdn.com/cartodb.js/v3/cartodb.js"></script>
        <script>

          function main() {

            // create leaflet map
            var map = L.map('map', { 
              zoomControl: false,
              scrollWheelZoom: false,
              center: [40.7127, -74.0059],
              zoomControl: true,
              zoom: 12
            })

            // add a base layer
            L.tileLayer('http://{s}.basemaps.cartocdn.com/dark_nolabels/{z}/{x}/{y}.png', {
              attribution: 'Carto'
            }).addTo(map);

            // add cartodb layer with one sublayer
            cartodb.createLayer(map, {
              user_name: "'''+username+'''",
              type: 'namedmap',
              named_map: {
                 name: "'''+map_name+'''",
                  layers: [{
                      layer_name: "layer0",
                      interactivity: "cartodb_id"
                  }]
              }
            })
            .addTo(map)
            .done(function(layer) {

              layer.getSubLayer(0).setInteraction(true)
              layer.getSubLayer(0).on('featureOver', function(e, pos, pixel, data) {
                console.log("Event #" + data.cartodb_id);
                // alert("Event #" + data.cartodb_id + ", magnitude " + data.mag);
              });
              cdb.vis.Vis.addInfowindow(map, layer.getSubLayer(0), ['cartodb_id'])
            });


          }

          // you could use $(window).load(main);
          window.onload = main; 

        </script>

      </body>
    </html>'''

In [144]:
text_file = open("index.html", "w")
text_file.write(html_text)
text_file.close()

[Links for Carto basemaps](http://bl.ocks.org/Xatpy/854297419bd7eb3421d0)

In [145]:
from IPython.display import display, HTML
display(HTML('<iframe width="100%" height="300" frameborder="0" src="index.html" allowfullscreen webkitallowfullscreen mozallowfullscreen oallowfullscreen msallowfullscreen></iframe>'))